***

**<center><font size = "6">Manage Data Models in Looker<center>**
***
<center><font size = "2">Prepared by: Sitsawek Sukorn<center>

### Modularizing LookML Code with Extends

### Extend a view to add columns from another view

#### Create a new view

- Click the toggle button to enter Development mode.

- On the Develop tab, select the qwiklabs-ecommerce LookML project.

- Click (+) next to File Browser, and select Create View.

- Name the view location, drag it under the views folder, and add the following code to it:

In [ ]:
view: location {
  extension: required
  dimension: city {
    type: string
    sql: ${TABLE}.city ;;
  }
  dimension: state {
    type: string
    sql: ${TABLE}.state ;;
    map_layer_name: us_states
  }
  dimension: zip {
    type: zipcode
    sql: ${TABLE}.zip ;;
  }
  dimension: country {
    type: string
    map_layer_name: countries
    sql: ${TABLE}.country ;;
  }
  
  dimension: latitude {
    type: number
    sql: ${TABLE}.latitude ;;
  }
  
  dimension: longitude {
    type: number
    sql: ${TABLE}.longitude ;;
  }
}

Notice line 2 (extension: required), which means that this view cannot be joined to other views, and thus will not be visible to users. To use this view, you must integrate it into another view using the extends parameter, which you do in the next section.

Also notice that, unlike with other views, you do not need to include the parameter sql_table_name in the view definition to identify which table to use for the data. Instead, this view will use the table specified in the view that will be extended in the next section.



- Click Save Changes, and then click Validate LookML. No LookML errors were found, and your file should resemble the following:

#### Add extends

- Open the users.view file.

- On a new line at the top of the file (line 1), add the following code, which indicates that the users view is being extended using the location view:

In [ ]:
include: location.view

- On line 3 above sql_table_name, add the following code:

In [ ]:
extends: [location]

Note: Because the extends are added, the sql_table_name parameter identifies which table to use as the data source for both the existing objects in the file and the objects that are added from geography via the extend.

- Remove the existing dimension definitions for: city, country, latitude, longitude, state, and zip (this is the existing order in the file). Instead of being explicitly defined in the users.view file, these dimensions are integrated via the extend from location.view.

- Click Save Changes, and then click Validate LookML.

- Open the event.view file.

- On a new line at the top of the file (line 1), add the following code:

In [ ]:
include: location.view

- On line 3 above sql_table_name, add the following code:

In [ ]:
extends: [location]

- As you did with the users view, remove the existing dimension definitions for: city, country, latitude, longitude, state, and zip.

! must move location.view to view if not that errors. in this lab not mention that.

#### Test the extended view for Users and Events in the Order Items Explore

- Navigate to the Explore page for Order Items.

- From the Users view, select the City, Country, Latitude, Longitude, State, and Zip dimensions.

- Click Run.

Even though you removed the definitions for these dimensions (city, country, latitude, longitude, state and zip) from the users.view file, you can see and use them because they were added to the users.view file using an extend from the location.view file!

- Navigate to the Events Explore.

- From the Events view, select the City, Country, Latitude, Longitude, State, and Zip dimensions.

Again, even though you removed the definitions for these dimensions from the events.view file, you can see and use them because they were added to the events.view file using an extend.

- Navigate back to the events.view file in the Looker IDE.

#### Commit changes and deploy to production


- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

### Extend an Explore to add joins from another Explore


- Navigate to the training_ecommerce.model file.

- After the order_items Explore definition (around line 43), create a new base Explore called base_events, using the following code:

In [ ]:
explore: base_events {
  extension: required
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }
}

Notice again the line for extension: required, which means that this Explore is not visible to users. Your file should resemble the following:

Next, you'll modify the existing definition for the events Explore to extend it with the views from base_events.

- From the events Explore, remove the existing joins for event_session_facts and users.

These joined views are integrated from the base_events Explore via code added in the next step. Notice that the join definition for event_session_funnel remains to customize this Explore for a particular set of users. Your file should resemble the following:

In [ ]:
explore: events {
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }

  to

  explore: events {
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }

- Under the first line of the events Explore definition, add the following code:

In [ ]:
description: "Start here for Event analysis"
  fields: [ALL_FIELDS*]
from: events
  view_name: events
  extends: [base_events]

#to

explore: events {
  description: "Start here for Event analysis"
  fields: [ALL_FIELDS*]
  from: events
  view_name: events
  extends: [base_events]
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }

Note: The new lines provide a description for the Explore info button, identify which fields from which view file to include (all fields), and specify which Explore is being used to extend the events Explore.

Your final definition for the events Explore should resemble the following:

In [ ]:
explore: events {
  description: "Start here for Event analysis"
  fields: [ALL_FIELDS*]
  from: events
  view_name: events
  extends: [base_events]
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
}

Note: The from and view_name are both pointing to the events view, so why include both? The from makes sure that you are using the original view called events (not an alias name for the view and not an extended one), and the view_name is the view file name, which could be an alias, etc.

- Below the modified events Explore definition, to add a new Explore called conversions, use the following code:

In [ ]:
explore: conversions {
  description: "Start here for Conversion Analysis"
  fields: [ALL_FIELDS*, -order_items.total_revenue_from_completed_orders]
  from: events
  view_name: events
  extends: [base_events]
  join: order_items {
    type: left_outer
    sql_on: ${users.id} = ${order_items.user_id} ;;
    relationship: many_to_many
  }
}

Lines 2-6 here provide a description for the Explore info button, identify which fields from which view file to include (all fields except the total_revenue_from_completed_orders measure in the order items view), and specify which Explore is being used to extend this Explore (i.e., the same base_events Explore that was used to extend the events Explore).

- Click Save Changes, and then click Validate LookML. No LookML errors were found, and your file should resemble the following:

Now it's time to test your new Explores. Go to each Explore (Events and Conversions), and notice which views are included. Because the Explores share a core set of views but are customized with additional views, each one serves a different user audience.

- Navigate to the Events Explore, which contains the views joined in the base Explore (Events, Event Session Facts, Users) plus the Event Session Funnel view.

- To review the description, hold the pointer over Information (info button) next to Events.

- Navigate to the Conversions Explore, which contains the views joined in the base Explore (Events, Event Session Facts, Users) plus the Order Items view.

- To review the description, hold the pointer over Information (info button) next to Conversions.

- Review the measures in the Order Items view; total_revenue_from_completed_orders is not listed.

- Return to the training_ecommerce.model file in the Looker IDE.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

***